Importing relevant packages from PyTorch

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 24 16:02:40 2021

@author: s2110992
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


Simulating Data

In [2]:
def random_rotation_matrix():
    theta = np.arccos(2*np.random.uniform(low = 0,high = 1)-1)
    phi = np.random.uniform(low = 0,high = 2*np.pi)
    u = np.array([np.sin(theta)*np.cos(phi),np.sin(theta)*np.sin(phi),np.cos(theta)])
    theta = np.random.uniform(low = 0,high = 2*np.pi)
    A = np.zeros((3,3))
    A[0][0] = np.cos(theta) + (u[0]**2)*(1-np.cos(theta))
    A[0][1] = u[0]*u[1]*(1-np.cos(theta)) - u[2]*np.sin(theta)
    A[0][2] = u[0]*u[2]*(1-np.cos(theta)) + u[1]*np.sin(theta)
    A[1][0] = u[1]*u[0]*(1-np.cos(theta)) + u[2]*np.sin(theta)
    A[1][1] = np.cos(theta) + (u[1]**2)*(1-np.cos(theta))
    A[1][2] = u[1]*u[2]*(1-np.cos(theta)) - u[0]*np.sin(theta)
    A[2][0] = u[2]*u[0]*(1-np.cos(theta)) - u[1]*np.sin(theta)
    A[2][1] = u[2]*u[1]*(1-np.cos(theta)) + u[0]*np.sin(theta)
    A[2][2] = np.cos(theta) + (u[2]**2)*(1-np.cos(theta))
    return A

In [3]:
def return_sphere_grid(r,m):
    
    theta = 2*np.pi*np.linspace(0,1,num=m)
    phi = np.arccos(1 - 2 * np.linspace(0,1,num=m))
    phi , theta = np.meshgrid(phi, theta)
    
    X = r*np.sin(phi) * np.cos(theta)
    Y = r*np.sin(phi) * np.sin(theta)
    Z = r*np.cos(phi)
#    print(np.shape(X))
    X = np.reshape(X,(m**2))
    Y = np.reshape(Y,(m**2))
    Z = np.reshape(Z,(m**2))
#    print(np.shape(X))
    
    data_sphere = np.array([X,Y,Z])
    return data_sphere

In [4]:
def return_ellipsoid_grid(m,x_stretch,y_stretch,z_stretch):
    
    theta = 2*np.pi*np.linspace(0,1,num=m)
    phi = np.arccos(1 - 2 * np.linspace(0,1,num=m))
    phi , theta = np.meshgrid(phi, theta)
    
    X = x_stretch*np.sin(phi) * np.cos(theta)
    Y = y_stretch*np.sin(phi) * np.sin(theta)
    Z = z_stretch*np.cos(phi)
#    print(np.shape(X))
    X = np.reshape(X,(m**2))
    Y = np.reshape(Y,(m**2))
    Z = np.reshape(Z,(m**2))
#    print(np.shape(X))
    
    data_ellipsoid= np.array([X,Y,Z])
    A = random_rotation_matrix()
    for i in range(m**2):
        data_ellipsoid[:,i] = np.matmul(A,data_ellipsoid[:,i])
    return data_ellipsoid

In [5]:
def return_sphere(r,m):
    noise_parameter = 0.05 #amount of noise away from a true sphere.
    x_eps = np.random.normal(size = m)*noise_parameter
    y_eps = np.random.normal(size = m)*noise_parameter
    z_eps = np.random.normal(size = m)*noise_parameter
    # create dataset
    
    theta = 2*np.pi*np.random.uniform(low=0,high=1,size=m)
    phi = np.arccos(1 - 2 * np.random.uniform(low=0,high=1,size=m))
    X = r*(1+x_eps)*np.sin(phi) * np.cos(theta)
    Y = r*(1+y_eps)*np.sin(phi) * np.sin(theta)
    Z = r*(1+y_eps)*np.cos(phi)
    
    data_sphere = np.array([X,Y,Z])
    return data_sphere
def return_ellipsoid(m,x_stretch,y_stretch,z_stretch):
    noise_parameter = 0.05 #amount of noise away from a true sphere.
    a_x = x_stretch#x-direction stretch
    b_y = y_stretch#y-direction stretch
    c_z  = z_stretch#z-direction stretch
    x_eps = np.random.normal(size = m)*noise_parameter
    y_eps = np.random.normal(size = m)*noise_parameter
    z_eps = np.random.normal(size = m)*noise_parameter
    # create dataset
    theta = 2*np.pi*np.random.uniform(low=0,high=1,size=m)
    phi = np.arccos(1 - 2 * np.random.uniform(low=0,high=1,size=m))
    X = a_x*(1+x_eps)*np.sin(phi) * np.cos(theta)
    Y = b_y*(1+y_eps)*np.sin(phi) * np.sin(theta)
    Z = c_z*(1+y_eps)*np.cos(phi)
    data_ellipsoid = np.array([X,Y,Z])
    A = random_rotation_matrix()
    for i in range(m):
        data_ellipsoid[:,i] = np.matmul(A,data_ellipsoid[:,i])
    return data_ellipsoid

In [6]:
training_data_size = 10000
group_size = training_data_size//2
data_size = 1024
#print(np.shape(return_sphere(100)))
training_set = np.zeros((training_data_size,3,data_size))
for i in range(group_size):
    training_set[i,:,:] = return_sphere(np.random.uniform(low = 1,high = 3),1024)
    stretch_factor = np.random.uniform(low = 20,high = 30)
    training_set[group_size+i,:,:] = return_ellipsoid_grid(1024,stretch_factor,1,1)
#training_set[:group_size,:] = np.random.uniform(size =(group_size,data_size))
#training_set[group_size:,:] = np.random.normal(size =(group_size,data_size))
#print(training_set[0])
training_set = np.random.permutation(training_set)
#print(training_set[0])
data_tensor = torch.FloatTensor(training_set)
#print(data_tensor[-1])
#x-stretch,y-stretch,z-stretch -> all in same data.

Training Data and Training Parameters

In [7]:
#Note we can make BATCH_SIZE smaller than training data set then it 
#does iterative gradient steps.
LEARNING_RATE = 1e-3
NUM_ITERS = 100
BATCH_SIZE = training_data_size

In [8]:
# Creating the architecture of the Neural Network

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder
        self.enc1 = nn.Linear(in_features=data_size, out_features=256)
        self.enc2 = nn.Linear(in_features = 256, out_features = 32)
        # decoder
        self.dec1 = nn.Linear(in_features = 32, out_features=256)
        self.dec2 = nn.Linear(in_features = 256, out_features =data_size)
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
    

        
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        return x
net = Autoencoder()
print(net)
net(torch.rand(3,data_size))

Autoencoder(
  (enc1): Linear(in_features=1024, out_features=256, bias=True)
  (enc2): Linear(in_features=256, out_features=32, bias=True)
  (dec1): Linear(in_features=32, out_features=256, bias=True)
  (dec2): Linear(in_features=256, out_features=1024, bias=True)
)


tensor([[0.0356, 0.0013, 0.0000,  ..., 0.0538, 0.0000, 0.0000],
        [0.0301, 0.0262, 0.0000,  ..., 0.0665, 0.0000, 0.0000],
        [0.0277, 0.0214, 0.0000,  ..., 0.0546, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)

In [9]:
#using Mean Square Error as the Loss function.
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [ ]:
#Training Step!.
loss_vector = np.zeros(NUM_ITERS)
for epoch in range(NUM_ITERS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(training_data_size):
        # get the inputs
        inputs = data_tensor[i]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        #summing loss to work out current loss.
        running_loss += loss.item()
    loss_vector[epoch] = running_loss
    if (abs(loss_vector[epoch] - loss_vector[epoch - 5])< 0.01):
        break
    #if epoch%10 ==0:
    print('running loss=',running_loss, 'epoch = ', epoch)
    
print('Finished Training')
low_dim_feat = net.enc2(F.relu(net.enc1(torch.rand(data_size))))


/home/s2110992/anaconda3/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


running loss= 221716.18848896027 epoch =  0
running loss= 210511.78508511186 epoch =  1
running loss= 206385.8709320724 epoch =  2
running loss= 203976.18825910985 epoch =  3
running loss= 202762.03527857363 epoch =  4
running loss= 198665.85215051472 epoch =  5
running loss= 195858.88681676984 epoch =  6
running loss= 194372.60432849824 epoch =  7


Cluster Feature Plot!

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
number_of_points = 1000

sphere_points = np.zeros((number_of_points,3,data_size))
ellipsoid_points = np.zeros((number_of_points,3,data_size))
sphere_labels = np.zeros(number_of_points)
ellipsoid_labels = np.zeros(number_of_points)

for i in range(number_of_points):
    r = np.random.uniform(low = 1,high = 3)
    sphere_points[i,:,:] = return_sphere(r,32)
    sphere_labels[i]= r
    stretch_factor = np.random.uniform(low = 20,high = 30)
    ellipsoid_points[i,:,:] = return_ellipsoid(32,stretch_factor,1,1)
    ellipsoid_labels[i] = stretch_factor
    

sphere_points = torch.FloatTensor(sphere_points)
#3d points each feature and see if they split up.


ellipsoid_points = torch.FloatTensor(ellipsoid_points)

sphere_feature1 = np.zeros(number_of_points)
sphere_feature2 = np.zeros(number_of_points)
sphere_feature3 = np.zeros(number_of_points)

ellipsoid_feature1 = np.zeros(number_of_points)
ellipsoid_feature2 = np.zeros(number_of_points)
ellipsoid_feature3 = np.zeros(number_of_points)

feature_number = 1
for i in range(number_of_points):
    sphere_feature = net.enc2(F.relu(net.enc1(sphere_points[i])))
    ellipsoid_feature = net.enc2(F.relu(net.enc1(ellipsoid_points[i])))
    sphere_feature= sphere_feature.detach().numpy()
    ellipsoid_feature = ellipsoid_feature.detach().numpy()
    sphere_feature1[i] = sphere_feature[0][feature_number]
    sphere_feature2[i] = sphere_feature[1][feature_number]
    sphere_feature3[i] = sphere_feature[2][feature_number]
    ellipsoid_feature1[i] = ellipsoid_feature[0][feature_number]
    ellipsoid_feature2[i] = ellipsoid_feature[1][feature_number]
    ellipsoid_feature3[i] = ellipsoid_feature[2][feature_number]
print(sphere_feature)
#m, b = np. polyfit(normal_feature1, normal_feature2, 1)  
plt.figure()
#3d plot
ax = plt.axes(projection='3d')

ax.scatter(sphere_feature1,sphere_feature2,sphere_feature3,label='Sphere')
ax.scatter(ellipsoid_feature1,ellipsoid_feature2,ellipsoid_feature3,label = 'Ellipsoid')




ax.set_title('Feature 1 of 32')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.legend()
plt.savefig('feature1-32-Nodes-Random.png')
#plt.figure()
#plt.plot(sphere_feature2,sphere_feature3,'r.',label = 'Sphere')
#plt.plot(ellipsoid_feature2,ellipsoid_feature3,'b.',label = 'Ellipsoid')


In [ ]:
plt.figure()
ax = plt.axes(projection='3d')



sc = ax.scatter(ellipsoid_feature1,ellipsoid_feature2,ellipsoid_feature3,c = ellipsoid_labels,label = 'Ellipsoid')
cm = plt.cm.get_cmap('RdYlBu')
ax.set_title('Ellipsoid colored according to Major Axis length')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
#ax.legend()
#loc = np.arange(0,max(label_sphere),max(label_sphere)/float(len(colors)))
#cb.set_ticks(loc)
#cb.set_ticklabels(colors)
#plt.scatter(ellipsoid_feature1,ellipsoid_feature2,'r.',label='Ellipsoids')
plt.colorbar(sc)
plt.show()
plt.savefig('feature1-32-Nodes-Ellipsoid-Stretch-Colour-.png')

In [ ]:
plt.figure()
ax = plt.axes(projection='3d')



sc = ax.scatter(sphere_feature1,sphere_feature2,sphere_feature3,c = sphere_labels,label='Sphere')
cm = plt.cm.get_cmap('RdYlBu')
ax.set_title('Spheres colored according to Radius')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
#ax.legend()
#loc = np.arange(0,max(label_sphere),max(label_sphere)/float(len(colors)))
#cb.set_ticks(loc)
#cb.set_ticklabels(colors)
#plt.scatter(ellipsoid_feature1,ellipsoid_feature2,'r.',label='Ellipsoids')
plt.colorbar(sc)
plt.show()
plt.savefig('feature1-32-Nodes-Sphere-Radii-Colour-Lattice.png')

Plot reconstructions!

In [ ]:
plt.figure()
bins = np.linspace(-800, 0, 800)
plt.hist(sphere_feature3,bins,color = 'blue', label = 'Sphere Data')
plt.hist(ellipsoid_feature3,bins,color = 'red',label = 'Ellipsoid Data')

plt.legend(loc='upper right')
plt.show() #make the bar sizes the same!


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
[X,Y,Z] = return_sphere_grid(2,32)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(X,Y,Z)

In [ ]:
sphere = [X,Y,Z]
sphere = torch.FloatTensor(sphere)
A = net(sphere)
[Xcon,Ycon,Zcon] = A.detach().numpy()
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(Xcon,Ycon,Zcon)

In [ ]:
stretch_factor = 8
[X,Y,Z] = return_ellipsoid(data_size,stretch_factor,1,1)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(X,Y,Z)

In [ ]:
ellipsoid = [X,Y,Z]
ellipsoid = torch.FloatTensor(ellipsoid)
A = net(ellipsoid)
[Xcon,Ycon,Zcon] = A.detach().numpy()
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(Xcon,Ycon,Zcon)